# Preliminary Exploratory Data Analysis

In [ ]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



The dataset can be read from the web to R as follows:

In [ ]:
htru_data <- read_csv(
    "https://github.com/Bruce0517/dsci-100-2022w1-group-160/raw/main/HTRU_2.csv", 
    col_names = c("p.mean", "p.sd", "p.kurt", "p.skew",
                  "c.mean", "c.sd", "c.kurt", "c.skew", "class"))

The data seems to describe some kind of distribution describing the data gathered from pulsar candidates. 0 means the candidate is not a pulsar, wihle 1 means the candidate is a pulsar. Here is a sample of some non-pulsars:

In [ ]:
set.seed(69420)
sample_n(filter(htru_data, class == 0), 3)

And here is a sample of some pulsars:

In [ ]:
set.seed(69420)
sample_n(filter(htru_data, class == 1), 3)

Then, we shall separate the data into training and testing sets:

In [ ]:
set.seed(69420)

htru_split <- initial_split(htru_data, prop = .8, strata = class)  
htru_train <- training(htru_split)   
htru_test <- testing(htru_split)

Here is the number of members of each class in the training data, as well as the average value they have for each variable:

In [ ]:
htru_train |>
    group_by(class) |>
    summarize(n = n(), p.mean = mean(p.mean), p.sd = mean(p.sd), p.kurt = mean(p.kurt), p.skew = mean(p.skew),
                       c.mean = mean(c.mean), c.sd = mean(c.sd), c.kurt = mean(c.kurt), c.skew = mean(c.skew))

As you can see, I did not need to include na.rm = TRUE, meaning that none of the rows have missing data. Here is a histogram for each of these values from a sample of 1000 pulsars and 1000 non-pulsars, to compare the distributions of the predictor variables:

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
set.seed(69420)

factor_levels <- c("p.mean", "p.sd", "p.kurt", "p.skew", "c.mean", "c.sd", "c.kurt", "c.skew")
pulsar_sample <- filter(htru_train, class == 1) |>
                     sample_n(1000) |>
                     # To plot the data we must first untidy the data
                     pivot_longer(-class, names_to = "preds", values_to = "vals") |>
                     mutate(preds = factor(preds, levels = factor_levels))
non_pulsar_sample <- filter(htru_train, class == 0) |>
                     sample_n(1000) |>
                     # To plot the data we must first untidy the data
                     pivot_longer(-class, names_to = "preds", values_to = "vals") |> 
                     mutate(preds = factor(preds, levels = factor_levels))
histo_data <- bind_rows(pulsar_sample, non_pulsar_sample)
facet_labels <- c("p.mean" = "Mean (Integrated Profile)", 
                  "p.sd" = "Standard Deviation (Integrated Profile)", 
                  "p.kurt" = "Excessive Kurtosis (Integrated Profile)", 
                  "p.skew" = "Skewness (Integrated Profile)",
                  "c.mean" = "Mean (DM-SNR Curve)", 
                  "c.sd" = "Standard Deviation (DM-SNR Curve)", 
                  "c.kurt" = "Excessive Kurtosis (DM-SNR Curve)", 
                  "c.skew" = "Skewness (DM-SNR Curve)")

ggplot(histo_data, aes(x = vals, fill = as_factor(class), color = as_factor(class))) +
    geom_histogram(alpha = 0.5, position = "identity", bins = 30) +
    facet_wrap(vars(preds), nrow = 4, ncol = 2,
               scales = "free", 
               labeller = as_labeller(facet_labels)) +
    labs(x = "Value", y = "Count", fill = "Class", color = "Class") +
    scale_fill_discrete(labels = c("Non-pulsar", "Pulsar")) +
    scale_color_discrete(labels = c("Non-pulsar", "Pulsar")) +
    theme(text = element_text(size=18))

The plots show that most values do not have a significant overlap for pulsars and non-pulsars, and thus all of them would be useful for data analysis.